# Applied Machine Learning: Real-World Classification and Model Exploration

The second part of the project applies machine learning techniques to a higher-dimensional, real-world dataset consisting of thousands of samples and hundreds of features. The task is to design, train, and evaluate a classification model capable of predicting discrete class labels from the dataset `datasetTV.csv`.

## Part A: Visualisation of the Dataset
The K-Means algorithm attempts to partition the dataset into five groups (clusters) based on the similarity of their observations. Its objective is to learn the underlying structure of the data and assign each record a label from 0 to 4, corresponding to the cluster ID. This ID is stored in a new column in the dataset named Cluster.

To visualize the features associated with each cluster, dimensionality reduction is required. For this purpose, Principal Component Analysis (PCA) is applied, retaining the two most significant components for two-dimensional representations and the three most significant components for three-dimensional representations.




In [1]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, accuracy_score

# Load dataset
dataset = np.loadtxt("datasetTV.csv", delimiter=",", skiprows=0)

# K-MEANS
kmeans = KMeans(n_clusters=5, random_state=42)
labels = kmeans.fit_predict(dataset)

# PCA 2D
pca_2d = PCA(n_components=2)
dataset_pca_2d = pca_2d.fit_transform(dataset)
fig_2d = px.scatter(
    x=dataset_pca_2d[:, 0],
    y=dataset_pca_2d[:, 1],
    color=(labels+1).astype(str),
    color_discrete_sequence=px.colors.qualitative.Prism,
    opacity=0.5,
    title="Clusters in 2D (PCA)",
)
fig_2d.update_layout(
    legend=dict(title="Class")
)
fig_2d.show()


# PCA 3D
pca_3d = PCA(n_components=3)
dataset_pca_3d = pca_3d.fit_transform(dataset)
fig_3d = px.scatter_3d(
    x=dataset_pca_3d[:, 0],
    y=dataset_pca_3d[:, 1],
    z=dataset_pca_3d[:, 2],
    color=(labels+1).astype(str),
    color_discrete_sequence=px.colors.qualitative.Prism,
    opacity=0.5,
    title="Clusters in 3D (PCA)",
)
fig_3d.update_layout(
    legend=dict(title="Class")
)
fig_3d.show()

The 3D PCA visualization demonstrates that K‑Means produces a largely well‑defined separation among the five clusters, with some groups forming dense and clearly distinguishable regions. A small central overlap indicates areas where class boundaries are less distinct, reflecting similarities in the underlying feature values. The dataset appears consistent and free of significant outliers, supporting strong model performance. The most challenging classification boundary emerges between clusters 1 and 4, whose samples occupy adjacent regions in PCA space and share similar feature profiles.

## Part B: Data Preprocessing

The dataset on which the model will be trained is loaded into the program via the Numpy library as a Numpy array, and array slicing is used to separate the features from the labels.

Subsequently, using the scikit-learn library, the dataset is split into a training set and a test set with an 80%-20% ratio. This is done so that it is possible to evaluate the model in terms of classification errors. The characteristics of the problem (large number of samples and features per sample, lack of knowledge of the physical meaning of the features, features that take continuous values) make data preprocessing necessary. All the preprocessing techniques used for this project (not all at the same time, not for every model) are described below.

- **Normalization (StandardScaler)**: This refers to the mathematical formulation
\begin{align}
x_{scaled} = \frac{x - μ_x}{σ_x}
\end{align}
which ensures that the data follow a Gaussian distribution with zero mean and unit variance $(~N(0,1))$. Normalization reduces bias toward specific features, since they now originate from the same distribution. As will be analyzed below, whether normalization is beneficial depends on the classifier that is chosen.

- **Normalization (MinMaxScaler)**: Since the dataset may contain columns with very different scales (e.g., one column with values 0–1 and another with 1000–5000), the Scaler transforms everything into the interval $[0, 1]$.
\begin{align}
x_{\text{scaled}} = a + \frac{(x - x_{\min})(b - a)}{x_{\max} - x_{\min}}
\end{align}
Without this, disproportionate importance would be assigned to larger values within the dataset while ignoring the smaller ones.

- **Principal Component Analysis (PCA)**: This is a linear dimensionality reduction method whose goal is to find components in the data that "capture" the greatest variance. The components that result from this process are orthogonal to each other (and therefore uncorrelated), with the first being the strongest (maximum variance) and each subsequent one capturing the remaining variance from the previous. Obviously, in order for the extraction of the components to be unbiased, data normalization must precede it. Dimensionality reduction lies in our freedom as model designers to choose a number of components that capture only a certain percentage of the variance. For this specific problem, we selected 70%, so that the reduction of the feature dimensionality from 224 to 88 is significant.

- **Validation Strategy (StratifiedKFold)**: During training preparation, the dataset is split into 5 segments (folds). The choice of "Stratified" ensures that each segment contains the same class proportions (e.g., 20% from each class), preventing the model from being trained in a biased way toward a particular group.


## Part C: Exploring Standard Classifiers

This section presents a range of classifiers in the attempt to explore which one is the most suitable for this particular problem. For every classifier in this section, the necessary preprocessing techniques are applied and explained, while the evaluation is executed based on the confusion matrix and the classification report offered by the `scikit-learn` library.

### 1. Maximum Likelihood Classifier (MLC):
The Maximum Likelihood Classifier is one of the most classical and widely used classification approaches, and it proves to be particularly effective when applied to large datasets.

Its functionality is based on the assumption that the data follow a multivariate Gaussian distribution (an assumption that constitutes the inductive bias introduced by this classifier) . Under the premise that the probability density function of the data for each class is Gaussian, the method estimates the parameters of these distributions (the mean vector and the covariance matrix of the features) in a parametric manner, selecting the values that maximize the likelihood function.

Once the probability density functions have been estimated in this way, the classifier can compute the posterior probability that a given sample belongs to a specific class, based on its feature values, by applying Bayes’ rule.

**Preprocessing:**
- Normalization using StandardScaler
- PCA capturing 70% of variance


In [8]:
features = dataset[:, 0:224]
labels = dataset[:, 224]

# Splitting -> training and test set
x_train, x_test, y_train, y_test = train_test_split(
    features, labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

# Scaling the training and testing features
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)

# Performing PCA on training and testing features
pca = PCA(
    n_components=0.8
)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Training MLC
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)

#Evaluating MLC
y_pred = qda.predict(X_test)
cm = confusion_matrix(y_pred, y_test)

class_names = ['1', '2', '3', '4', '5']
fig = px.imshow(
    cm,
    text_auto=True,
    width=800,
    color_continuous_scale="Agsunset",
    labels=dict(x="Predicted Label", y="True Label", color="Count"),
    x=class_names,
    y=class_names,
)
fig.update_layout(
    title=f"Confusion Matrix: Maximum Likelihood Classifier",
    xaxis=dict(side="bottom")
)
fig.show()
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.97      0.97      0.97       354
         2.0       0.72      0.69      0.70       344
         3.0       0.93      0.91      0.92       351
         4.0       0.89      0.90      0.90       343
         5.0       0.72      0.76      0.74       357

    accuracy                           0.85      1749
   macro avg       0.85      0.85      0.85      1749
weighted avg       0.85      0.85      0.85      1749



**Results: Accuracy = 85.23%**

Based on the results, the model achieves an overall accuracy of approximately 85%, indicating strong general performance on the 5‑class classification task. The classification report shows that classes 1 and 3 are classified exceptionally well, with very high precision and recall (≈0.93–0.97), while class 4 also demonstrates balanced and solid performance. This suggests that the model reliably identifies samples from these categories and makes few errors.

Classes 2 and 5 exhibit lower performance, particularly the recall of class 2 (0.69), indicating that many samples from this class are confused with others—mainly with class 5, as also reflected in the confusion matrix. The macro and weighted averages are similar, suggesting that the dataset is relatively balanced.



### 2. k-Nearest Neighbors

The k‑Nearest Neighbors (kNN) classifier is one of the most intuitive algorithms and belongs to the family of non‑parametric learning methods.

The inductive bias introduced by this classifier is the assumption of locality—the idea that samples located close to one another in the feature space are likely to belong to the same class. As a non‑parametric method, kNN does not assume any specific form for the underlying probability density function of the data, which allows it to adapt to complex and non‑linear patterns without the need to estimate distributional parameters.

Its functionality is based on computing a distance function—typically the Euclidean distance—between the unknown sample and all available training samples. The final decision follows the majority‑vote principle: the sample is assigned to the class most common among its k nearest neighbors. Because this classifier relies heavily on distance calculations, data normalization is essential.

The choice of $k$ plays a critical role in the classifier’s performance. Large values of k may lead to oversmoothing and models that are overly sensitive to noise, whereas very small values may produce decision boundaries that fail to generalize well, resulting in underfitting.

**Preprocessing:**
- Normalization using StandardScaler
- PCA capturing 80% of variance

**Hyperparameters:**
- **n_neighbors = 7:** the number of nearest neighbors considered during classification.
- **weights = 'distance':** determines each neighbor’s contribution to the decision (with 'distance', closer neighbors exert greater influence than more distant ones).
- **metric = 'minkowski':** the distance metric used to compute neighbor proximity (the Minkowski metric corresponds to Euclidean distance for the standard parameter settings).

In [9]:
# Same loading and preprocessing as before#

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(
    n_neighbors=7,
    weights='distance',
    metric='minkowski',
)
knn.fit(X_train, y_train)

# Evaluating kNN
y_pred = knn.predict(X_test)
cm = confusion_matrix(y_pred, y_test)

fig = px.imshow(
    cm,
    text_auto=True,
    width=800,
    color_continuous_scale="Agsunset",
    labels=dict(x="Predicted Label", y="True Label", color="Count"),
    x=class_names,
    y=class_names,
)
fig.update_layout(
    title=f"Confusion Matrix: k-NN Classifier",
    xaxis=dict(side="bottom")
)
fig.show()
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.93      0.96      0.95       354
         2.0       0.76      0.56      0.65       344
         3.0       0.91      0.92      0.91       351
         4.0       0.94      0.89      0.91       343
         5.0       0.67      0.85      0.75       357

    accuracy                           0.84      1749
   macro avg       0.84      0.84      0.83      1749
weighted avg       0.84      0.84      0.83      1749



**Results: Accuracy = 83.79%**

The k‑Nearest Neighbors (k‑NN) classifier with $𝑘=7$
 and Euclidean distance achieves an overall accuracy of approximately 83.6%, which is slightly lower than that of the Maximum Likelihood classifier. This suggests that the latter generalizes more effectively for the present classification task. As in previous results, classes 1, 3, and 4 exhibit strong performance, with high precision and recall, indicating that these classes are well separated in the feature space. In contrast, class 2 is frequently misclassified as class 5, a pattern clearly reflected in the confusion matrix. This behavior highlights k‑NN’s sensitivity to local class overlap, since its decisions rely directly on neighborhood structure and distance metrics.

### 3. Logistic Regression

Logistic Regression for multiclass classification belongs to the family of discriminative probabilistic models. For each of the five classes, the method defines a linear function (a weighted linear combination of the input features), which is then transformed into a valid probability distribution—non‑negative values that sum to one—through the Softmax function
\begin{align}
P(ω_i|x) = \frac{exp(w_i^T)}{\sum{exp(w_i^Tx)}}
\end{align}
The model parameters are learned using Maximum Likelihood estimation, optimized via Stochastic Gradient Descent (SGD) with the Cross‑Entropy Loss as the objective function. During inference, a new sample
 is assigned to the class with the highest posterior probability produced by the model.

**Preprocessing:**
- Normalization using StandardScaler
- PCA capturing 80% of variance


In [10]:
# Same preprocessing as before

# Train the model
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)

# Evaluate the model
y_pred = lr.predict(X_test)
cm = confusion_matrix(y_pred, y_test)
fig = px.imshow(
    cm,
    text_auto=True,
    width=800,
    color_continuous_scale="Agsunset",
    labels=dict(x="Predicted Label", y="True Label", color="Count"),
    x=class_names,
    y=class_names,
)
fig.update_layout(
    title=f"Confusion Matrix: Logistic Regression",
    xaxis=dict(side="bottom")
)
fig.show()
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

         1.0       0.92      0.95      0.93       354
         2.0       0.71      0.65      0.68       344
         3.0       0.83      0.83      0.83       351
         4.0       0.82      0.85      0.83       343
         5.0       0.69      0.71      0.70       357

    accuracy                           0.80      1749
   macro avg       0.79      0.80      0.79      1749
weighted avg       0.79      0.80      0.79      1749



**Results: Accuracy = 79.84%**

Logistic Regression exhibits relatively low overall accuracy, indicating that this linear model struggles to fully capture the underlying structure of the problem. Although class 1 continues to be classified with strong performance, a general decline in both precision and recall is observed across the remaining classes. Class 2, in particular, again shows a notably low recall (0.63), with the confusion matrix revealing substantial misclassification into class 5 as well as confusion with other classes.

Classes 3 and 4 demonstrate moderate yet stable performance, though they do not reach the levels achieved by the previous models. These results suggest that Logistic Regression is less capable of handling non‑linear decision boundaries in the feature space. Overall, the findings indicate that the task requires more flexible models, as the linear assumptions inherent to Logistic Regression limit its final performance.

### 4. Random Forests

Random Forests are ensemble methods that combine the predictions of multiple decision trees for classification or regression tasks. Each tree is trained on a different bootstrap sample of the dataset (bagging), and at every split only a randomly selected subset of features is considered. This injected randomness reduces correlation among the trees and helps mitigate overfitting.

In multiclass classification, each tree outputs a class prediction, and the final decision of the forest is obtained through majority voting. Random Forests are robust to noise, perform well without extensive hyperparameter tuning, and can effectively handle high‑dimensional data—hence **their training is performed directly on the original rather than the preprocessed feature set**.

**Hyperparameters:**
- n_estimators: the number of decision trees included in the forest
- criterion: the measure used to evaluate the quality of a split (e.g., gini, which favors purer nodes)
- max_features: the number of features considered at each split (using sqrt reduces tree correlation and helps control overfitting)
- random_state: the random seed ensuring reproducible results

In [39]:
# No preprocessing required

# Train the model
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(
    n_estimators=270,
    criterion='gini',
    max_features='sqrt',
    random_state=42,
)
model.fit(x_train, y_train)

# Evaluate the model
y_pred= model.predict(x_test)
cm = confusion_matrix(y_pred, y_test)
fig = px.imshow(
    cm,
    text_auto=True,
    width=800,
    color_continuous_scale="Agsunset",
    labels=dict(x="Predicted Label", y="True Label", color="Count"),
    x=class_names,
    y=class_names,
)
fig.update_layout(
    title=f"Confusion Matrix: Random Forest Classifier",
    xaxis=dict(side="bottom")
)
fig.show()
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.94      0.98      0.96       354
         2.0       0.75      0.69      0.72       344
         3.0       0.90      0.93      0.91       351
         4.0       0.91      0.88      0.89       343
         5.0       0.74      0.76      0.75       357

    accuracy                           0.85      1749
   macro avg       0.85      0.85      0.85      1749
weighted avg       0.85      0.85      0.85      1749



**Results: Accuracy = 84.91%**

The Random Forest classifier achieves an accuracy of approximately 84.9%, placing its performance very close to that of the Maximum Likelihood model. The strong results for classes 1 and 3 (f1‑score ≥ 0.91) indicate that the model effectively captures non‑linear relationships and interactions among the 224 features. However, as observed with previous classifiers, classes 2 and 5 remain the most challenging, with substantial confusion between them in the confusion matrix. Overall, the Random Forest demonstrates good balance and robustness, providing a satisfactory solution to the classification problem.

### 5. AdaBoost Classifier

AdaBoost is an ensemble method that combines multiple weak learners—typically decision stumps—to construct a strong classifier. It operates by training a sequence of models in which each new learner places greater emphasis on the samples that were misclassified in previous rounds, achieved through adaptive reweighting of the training instances.

Each classifier receives a weight proportional to its accuracy, and the final prediction is obtained through a weighted voting scheme. AdaBoost can achieve high accuracy and, similar to Random Forests, **tends to perform best when trained on the original rather than the preprocessed feature set.**

**Hyperparameters:**
- **max_depth = 5:** the maximum depth of each tree; with depth 5, a tree may perform up to five splits before reaching a decision
- **min_samples_leaf = 40:** the minimum number of samples required in each leaf node, helping to prevent overfitting
- **random_state = 42:** the random seed ensuring reproducible results
- **estimator = base_tree:** specifies the decision tree used as the base learner
- **n_estimators = 180:** the number of boosting rounds; the model constructs 180 trees sequentially, with each tree attempting to correct the errors of its predecessors
- **learning_rate = 0.12:** controls the contribution of each tree to the final model; a lower learning rate leads to slower, more conservative learning, typically requiring more estimators but often improving generalization

In [40]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

base_tree = DecisionTreeClassifier(
    max_depth=5,      # decision stump
    min_samples_leaf=50,
    random_state=42
)

ada = AdaBoostClassifier(
    estimator=base_tree,   # weak learner
    n_estimators=180,      # number of boosting rounds
    learning_rate=0.15, # contribution of each weak learner
    random_state=42
)

# Train and evaluate
ada.fit(x_train, y_train)
y_pred = ada.predict(x_test)
cm = confusion_matrix(y_pred, y_test)
fig = px.imshow(
    cm,
    text_auto=True,
    width=800,
    color_continuous_scale="Agsunset",
    labels=dict(x="Predicted Label", y="True Label", color="Count"),
    x=class_names,
    y=class_names,
)
fig.update_layout(
    title=f"Confusion Matrix: AdaBoost Classifier",
    xaxis=dict(side="bottom")
)
fig.show()
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.97      0.96      0.96       354
         2.0       0.71      0.70      0.70       344
         3.0       0.92      0.89      0.91       351
         4.0       0.92      0.87      0.90       343
         5.0       0.69      0.76      0.72       357

    accuracy                           0.84      1749
   macro avg       0.84      0.84      0.84      1749
weighted avg       0.84      0.84      0.84      1749

0.8359062321326473


**Results: Accuracy = 83.59%**

The AdaBoost classifier achieves an overall accuracy of approximately 83.6%, comparable to that of k‑NN and slightly below the performance of Random Forests and the Maximum Likelihood classifier. Its exceptionally strong performance on class 1 (f1‑score ≈ 0.96) indicates that boosting effectively strengthens weak learners for well‑separated classes. Classes 3 and 4 also exhibit robust and stable behavior.

Classes 2 and 5 however remain the most challenging, showing substantial bidirectional confusion in the confusion matrix—a pattern consistently observed across all classifiers. Compared with Random Forests, AdaBoost appears more sensitive to difficult or noisy samples, which is expected given its iterative emphasis on correcting previous errors.

### 6. Multi-Layer Perceptron (MLP)

The MLP consists of an input layer, a single hidden layer, and an output layer where classification takes place.

**Preprocessing:**
- Scaling using MinMaxScaler
- PCA capturing 80% of variance


**Hyperparameters:**

- **hidden_layer_sizes = [(400,)]:** Defines the network architecture, here a single hidden layer with 400 neurons.
- **activation = 'relu':** The activation function determines whether a neuron fires (i.e., transmits a signal to the next layer). Mathematically, ReLU is defined as
$$f(x)=max(0, x)$$
It enables the network to learn non‑linear relationships and alleviates the vanishing‑gradient problem.
- **solver = 'adam':** The optimization algorithm that updates the network’s weights to minimize prediction error. Adam adapts the learning rate for each parameter individually to efficiently approach the global minimum of the loss function.
- **learning_rate_init = 0.001:** The initial step size used by Adam during weight updates.
- **max_iter = 1055:** Specifies the number of epochs, i.e., how many full passes through the dataset the network performs during training.

In [13]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier

# Scaling with Min-Max Scaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)

# PCA Transformation
pca = PCA(n_components=0.8)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Training the MLP Classifier
mlp = MLPClassifier(
    hidden_layer_sizes=(300,),   # you can change this
    activation='relu',
    solver='adam',
    learning_rate_init=0.001,
    max_iter=1200,
    random_state=42
)
mlp.fit(X_train, y_train)


# Evaluation
y_pred = mlp.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
class_names = ['1', '2', '3', '4', '5']
fig = px.imshow(
    cm,
    text_auto=True,
    width=800,
    color_continuous_scale="Agsunset",
    labels=dict(x="Predicted Label", y="True Label", color="Count"),
    x=class_names,
    y=class_names,
)
fig.update_layout(
    title="Confusion Matrix: MLP Classifier",
    xaxis=dict(side="bottom")
)
fig.show()
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))


              precision    recall  f1-score   support

         1.0       0.97      0.97      0.97       354
         2.0       0.81      0.74      0.77       344
         3.0       0.94      0.95      0.94       351
         4.0       0.92      0.93      0.93       343
         5.0       0.78      0.83      0.80       357

    accuracy                           0.88      1749
   macro avg       0.88      0.88      0.88      1749
weighted avg       0.88      0.88      0.88      1749

0.8827901658090337


**Results: Accuracy = 88.28%**

The performance analysis of the MLP Classifier reveals a balanced and realistic assessment of the model’s capabilities, with  accuracy reaching 88.28%. Once again, the confusion matrix highlights the network’s primary challenge: the incomplete separation of classes 2 and 5. Class 2 exhibits the lowest recall (0.74), which is consistent with the visual overlap observed in the 3D PCA plot. In contrast, classes 1, 3, and 4 achieve exceptionally high success rates, reflecting their clearer separability in the principal‑component space.

### 7. Naive Bayes Classifier

Naive Bayes
Naive Bayes is based on Bayes’ Theorem, which enables the computation of the probability that a sample belongs to a particular class given its observed features. Its mathematical formulation is:
$$P(C|x) = \frac{P(x|C) \cdot P(C)}{P(x)}$$
Where:

- $P(C|x)$: the probability that the sample $x$ belongs to class $C$
- $P(x|C)$:the likelihood of observing the data $x$ assuming the sample originates from class $C$.
- $P(C)$: the prior probability of class $C$$ in the dataset.

The algorithm assumes feature independence and therefore requires only the estimation of the mean and variance of each feature for each class. Once these quantities are computed, the model is fully specified.

**Preprocessing:**
- Scaling using MinMaxScaler
- PCA capturing 80% of variance


In [7]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(X_train, y_train)

# Evaluation
y_pred = nb.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
class_names = ['1', '2', '3', '4', '5']
fig = px.imshow(
    cm,
    text_auto=True,
    width=800,
    color_continuous_scale="Agsunset",
    labels=dict(x="Predicted Label", y="True Label", color="Count"),
    x=class_names,
    y=class_names,
)
fig.update_layout(
    title="Confusion Matrix: Naive Bayes Classifier",
    xaxis=dict(side="bottom")
)
fig.show()
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.92      0.95      0.94       354
         2.0       0.70      0.57      0.63       344
         3.0       0.81      0.81      0.81       351
         4.0       0.82      0.83      0.83       343
         5.0       0.65      0.73      0.69       357

    accuracy                           0.78      1749
   macro avg       0.78      0.78      0.78      1749
weighted avg       0.78      0.78      0.78      1749

0.779874213836478


**Results: Accuracy = 77.99%**

The maximum accuracy achieved is approximately 78%. Since Naive Bayes assumes independence among features, it is unable to capture the complex relationships that characterize the overlap between classes 2 and 5. Class 2, in particular, attains an accuracy of only 57%, with the confusion matrix revealing said overlap. Nevertheless, the classifier performs exceptionally well on class 1, which appears to be the most statistically separable class in the dataset.

### 8. Support Vector Machine (SVM) with RBF Kernel

SVM tries to find the optimal margin, that is, the largest possible distance between the classes. Because the data (as we saw in the 3D plot) cannot be separated with a simple straight line, the RBF kernel allows the model to create non‑linear and curved boundaries.
The RBF Kernel computes the proximity or similarity between two points $x_1$ and $x_2$
$$K(x_1, x_2) = \exp\left(-\gamma \cdot \|x_1 - x_2\|^2\right)$$

- Distance: The closer two points are in space, the higher the Kernel value (approaching 1).
- Radial Structure: The name "Radial" comes from the fact that the influence of each point spreads circularly around it, like a bell (Gaussian distribution).

It maps the data into a space where each sample has an infinite number of features, making them linearly separable. It uses the Kernel Trick: it directly computes the “similarity” (inner product) of points in the original space, but the result is as if the computation had been done in infinite dimensions.

**Preprocessing:**
- Scaling using MinMaxScaler
- PCA capturing 80% of variance

**Hyperparameters:**
- **gamma='scale':** Determines how “far” the influence of a single sample reaches. The value 'scale' automatically computes this based on the variance of the data.
- **C=2.0:**  The regularization parameter. It controls the balance between correctly classifying the training data and keeping the boundary simple, to avoid overfitting and achieve better generalization on the unknown test data.

In [14]:
from sklearn.svm import SVC

# Training the SVM Classifier
svm = SVC(
    kernel='rbf',        # you can change to 'linear', 'poly', etc.
    C=3.0,               # regularization strength
    gamma='scale',       # kernel coefficient
    random_state=42
)
svm.fit(X_train, y_train)

# Evaluation
y_pred = svm.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
class_names = ['1', '2', '3', '4', '5']
fig = px.imshow(
    cm,
    text_auto=True,
    width=800,
    color_continuous_scale="Agsunset",
    labels=dict(x="Predicted Label", y="True Label", color="Count"),
    x=class_names,
    y=class_names,
)
fig.update_layout(
    title="Confusion Matrix: SVM with RBF Kernel",
    xaxis=dict(side="bottom")
)
fig.show()
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.97      0.98      0.98       354
         2.0       0.82      0.72      0.77       344
         3.0       0.94      0.96      0.95       351
         4.0       0.95      0.94      0.94       343
         5.0       0.78      0.85      0.81       357

    accuracy                           0.89      1749
   macro avg       0.89      0.89      0.89      1749
weighted avg       0.89      0.89      0.89      1749

0.8902229845626072


**Results: Accuracy = 89.02%**

This is the strongest classifier for this problem so far, achieving the maximum accuracy

### 9. SVM with Polynomial Kernel

The polynomial kernel tries to find separating boundaries that resemble higher‑order curves (such as parabolas or cubic curves). Essentially, the algorithm examines not only the original features of the data, but also their combinations. This allows it to detect interactions between the columns of datasetTV.csv that a simple linear classifier would miss.

**Preprocessing:**
- Scaling using MinMaxScaler
- PCA capturing 80% of variance

**Hyperparameters:**
- **degree=3:** The degree of the polynomial. In our case it is 3 (cubic polynomial). It means that the model can “see” relationships between three features simultaneously. A small and odd polynomial degree was chosen to avoid overfitting but also to prevent loss of sign information (because after PCA some principal components have negative values), since the model would otherwise exhibit a “mirror‑like” behavior and consider points that are equally distant from the center (either left or right) as similar.
- **C=1.5:** The model tries to correctly classify the data but allows some errors so that the separating curve remains smooth, meaning that as before we selected soft margin. From experimentation with the values, this was considered the best choice because anything higher led to overfitting.

In [15]:
# Training the SVM Classifier
svm = SVC(
    kernel='poly',
    C=2.0,
    degree=3,
    random_state=42
)
svm.fit(X_train, y_train)

# Evaluation
y_pred = svm.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
class_names = ['1', '2', '3', '4', '5']
fig = px.imshow(
    cm,
    text_auto=True,
    width=800,
    color_continuous_scale="Agsunset",
    labels=dict(x="Predicted Label", y="True Label", color="Count"),
    x=class_names,
    y=class_names,
)
fig.update_layout(
    title="Confusion Matrix: SVM with Poly Kernel",
    xaxis=dict(side="bottom")
)
fig.show()
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.97      0.97      0.97       354
         2.0       0.79      0.73      0.76       344
         3.0       0.93      0.95      0.94       351
         4.0       0.96      0.93      0.95       343
         5.0       0.77      0.84      0.80       357

    accuracy                           0.88      1749
   macro avg       0.89      0.88      0.88      1749
weighted avg       0.89      0.88      0.88      1749

0.8839336763865066


**Results: Accuracy = 88.39%**

The performance of the SVM with the polynomial kernel is comparable to the SVM with the RBF kernel. These classifiers seem to be the most suitable for the problem in question, and will therefore constitute the basis of the experimentation section following

## Part D: Custom Classifier Experimentation

### Feature Extraction with MLP and Classification with SVM
An experiment was conducted using a neural network (MLP) as a feature extractor (Deep Feature Extraction). The model was trained to transform the original 224 features into a 128-dimensional vector through a dense layer.

### Data Preprocessing
- The training dataset was split into 80% training samples and 20% validation samples, using stratify to ensure that the class distribution (e.g., how many samples belong to Class 2 and how many to Class 5) remains the same in both sets, thus avoiding model bias.
- Scaling was applied using **MinMaxScaler** in the range [0,1], because neural networks are sensitive to large values.
- **One-hot encoding** was used to convert class numbers into binary vectors. This is necessary for the final layer of the Neural Network (Softmax layer), which computes probabilities separately for each class.

### Architecture and Training
- **Input and hidden layers:** The network receives 224 inputs (input_dim), corresponding to the features of the data, whereas the hidden layers consist of 512, 256 and 128 neurons respectively
- **Activation = 'relu':** Allows the network to learn non-linear relationships.
- **Kernel Initializer = 'he_normal':** An optimal weight initialization method for ReLU layers, which helps with training stability and faster convergence.
- **Dropout Layer:** At each training step, 45% of the neurons are randomly deactivated, forcing the network not to rely on specific neurons and thus generalize better (avoiding overfitting).
- **Output layer:** It consists of 5 neurons, equal to the number of problem classes. The activation function is softmax, which converts outputs into probabilities that sum to 1.0 (e.g., 80% probability for Class 1).
- **Loss function:** Cross Entropy Function for multi-class classification problems when using one-hot encoding. It measures the "distance" between the prediction and the true class.
- **Adam Optimizer:** The algorithm that determines how the neuron weights are updated in order to reduce prediction error. It adapts the learning rate for each weight individually to find the global minimum of the loss function.
- **Early Stopping:** If the validation set accuracy (val_accuracy) stops improving for 25 consecutive epochs, training stops automatically.
- **Batch Size:** The model processes 70 samples at a time before updating its weights.
- Training is set for 100 epochs; however, it stops earlier due to the early stopping mechanism.

### Classification with SVM
The penultimate layer (the last_dropout) contains the essence of the information in a form that the network considers ideal for distinguishing between classes. The original 224 features of `datasetTV.csv` are transformed into 128 new features (equal to the neurons of the Dense layer). These are fed into an SVM with an RBF kernel, with an increased regularization parameter that allows it to draw highly complex and non-linear decision boundaries in this new high-dimensional space, achieving a stricter and more accurate class separation compared to the probabilistic approaches of MLPs (from the softmax). This geometric approach to classification, combined with the automatic setting gamma='scale', allows the model to exploit the variance of the 420 neurons to resolve ambiguities in regions of strong overlap—such as between classes 2 and 5 (as also shown in the confusion matrix)—leading to a validation set accuracy of 87.25%. We observe that the SVM accuracy is 2.5% higher than that of the MLP model. This indicates that the last layer of the MLP has learned an improved feature representation, while the SVM attempts to capture even at this level the non-linear relationships.

In [19]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly.graph_objects as go
import plotly.express as px

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

# Reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

# 2. Load Data
print("Loading data...")
train_df = pd.read_csv('datasetTV.csv', header=None)
test_submit_df = pd.read_csv('datasetTest.csv', header=None)

X = train_df.iloc[:, :-1].values
y = train_df.iloc[:, -1].values

# Convert labels 1-5 → 0-4
y_adjusted = y - 1

# Train / Validation Split
x_train, x_val, y_train, y_val = train_test_split(
    X,
    y_adjusted,
    test_size=0.2,
    stratify=y_adjusted,
    random_state=SEED
)

# Scale for neural network training
nn_scaler = MinMaxScaler()
x_train = nn_scaler.fit_transform(x_train)
x_val = nn_scaler.transform(x_val)
x_test_submit = nn_scaler.transform(test_submit_df.values)

y_train_onehot = to_categorical(y_train, 5)
y_val_onehot = to_categorical(y_val, 5)

# MLP Feature Extractor
print("Training MLP Feature Extractor...")
mlp = Sequential([
    Dense(512, activation='relu', kernel_initializer='he_normal', input_dim=224),
    BatchNormalization(),
    Dropout(0.4),

    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(128, activation='relu', name='feature_layer'),
    Dropout(0.2),

    Dense(5, activation='softmax')
])
mlp.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True
)
lr_reduce = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=7,
    min_lr=1e-6,
    verbose=1
)
history = mlp.fit(
    x_train,
    y_train_onehot,
    validation_data=(x_val, y_val_onehot),
    epochs=100,
    batch_size=64,
    callbacks=[early_stopping, lr_reduce],
    verbose=2
)

# Feature Extraction
feature_extractor = Model(
    inputs=mlp.inputs,
    outputs=mlp.get_layer("feature_layer").output
)
train_features = feature_extractor.predict(x_train)
val_features = feature_extractor.predict(x_val)
test_features = feature_extractor.predict(x_test_submit)
print("Extracted feature shape:", train_features.shape)

# sklearn Pipeline + GridSearchCV (SVM Head)
print("Running GridSearch on SVM head...")
svm_pipeline = Pipeline([
    ('scaler', MinMaxScaler()),   # scaling for SVM
    ('svm', SVC())
])
param_grid = {
    'svm__C': [0.1, 1, 5, 10, 50],
    'svm__gamma': ['scale', 0.1, 0.05, 0.02, 0.01, 0.001, 0.0001],
    'svm__kernel': ['rbf']
}
grid = GridSearchCV(
    svm_pipeline,
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)
grid.fit(train_features, y_train)
print("Best Parameters:", grid.best_params_)
print("Best CV Accuracy:", grid.best_score_)

# Validation Evaluation
best_model = grid.best_estimator_
val_predictions = best_model.predict(val_features)
val_acc = accuracy_score(y_val, val_predictions)
print(f"\nFinal Validation Accuracy: {val_acc * 100:.2f}%")
cm = confusion_matrix(y_val, val_predictions)
class_names = ['1', '2', '3', '4', '5']
fig = px.imshow(
    cm,
    text_auto=True,
    width=800,
    color_continuous_scale="Agsunset",
    labels=dict(x="Predicted Label", y="True Label", color="Count"),
    x=class_names,
    y=class_names,
)
fig.update_layout(
    title="Confusion Matrix: MLP with SVM Head",
    xaxis=dict(side="bottom")
)
fig.show()
print(classification_report(y_val, val_predictions))
print(accuracy_score(y_val, val_predictions))

print("Plotting MLP Training Curves...")

epochs_range = list(range(1, len(history.history['accuracy']) + 1))

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=epochs_range,
    y=history.history['accuracy'],
    mode='lines',
    name='MLP Train Accuracy'
))

fig.add_trace(go.Scatter(
    x=epochs_range,
    y=history.history['val_accuracy'],
    mode='lines',
    name='MLP Validation Accuracy'
))

fig.update_layout(
    title="MLP Feature Learning Progress",
    xaxis_title="Epochs",
    yaxis_title="Accuracy",
    width=900,
    height=500,
)

Loading data...
Training MLP Feature Extractor...
Epoch 1/100


C:\Users\samam\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:106: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



110/110 - 2s - 18ms/step - accuracy: 0.6231 - loss: 1.0159 - val_accuracy: 0.6627 - val_loss: 0.8955 - learning_rate: 0.0010
Epoch 2/100
110/110 - 1s - 5ms/step - accuracy: 0.7449 - loss: 0.6528 - val_accuracy: 0.7810 - val_loss: 0.6098 - learning_rate: 0.0010
Epoch 3/100
110/110 - 1s - 5ms/step - accuracy: 0.7771 - loss: 0.5652 - val_accuracy: 0.8176 - val_loss: 0.4597 - learning_rate: 0.0010
Epoch 4/100
110/110 - 1s - 5ms/step - accuracy: 0.8010 - loss: 0.4918 - val_accuracy: 0.8262 - val_loss: 0.4268 - learning_rate: 0.0010
Epoch 5/100
110/110 - 0s - 4ms/step - accuracy: 0.8194 - loss: 0.4446 - val_accuracy: 0.8330 - val_loss: 0.4181 - learning_rate: 0.0010
Epoch 6/100
110/110 - 0s - 4ms/step - accuracy: 0.8337 - loss: 0.4141 - val_accuracy: 0.8399 - val_loss: 0.3965 - learning_rate: 0.0010
Epoch 7/100
110/110 - 0s - 4ms/step - accuracy: 0.8430 - loss: 0.3911 - val_accuracy: 0.8582 - val_loss: 0.3704 - learning_rate: 0.0010
Epoch 8/100
110/110 - 0s - 4ms/step - accuracy: 0.8494 - lo

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       354
           1       0.79      0.73      0.76       344
           2       0.93      0.95      0.94       351
           3       0.93      0.91      0.92       343
           4       0.76      0.81      0.78       357

    accuracy                           0.87      1749
   macro avg       0.88      0.87      0.87      1749
weighted avg       0.88      0.87      0.87      1749

0.8747855917667239
Plotting MLP Training Curves...




### MLP Learning Curves
In the first 20–30 epochs, we observe a sharp increase in both curves (Train and Val Acc), indicating that the model quickly identifies the fundamental data patterns. The Validation Accuracy curve stabilizes around epoch 40, without exceeding the 85% threshold. This shows that the standalone MLP is limited to this performance due to the complexity of the problem. After epoch 40, training accuracy continues rising toward 0.90, while validation accuracy remains stagnant. The validation accuracy exhibits strong spikes (noise). This suggests that the Adam algorithm struggles to find a stable minimum in the overlapping region of classes 2 and 5, something that the SVM with the RBF kernel handles much more smoothly.